In [1]:
from acevedo_clss_and_fcns import * 
device = 'cpu'
if torch.cuda.is_available():
    torch.cuda.init()
    if torch.cuda.is_initialized():
        device = 'cuda:0'
#device = torch.device(device)
print(f"{device = }")
from torch_geometric.nn import GNNExplainer

    

model = torch.load("results/trained_pytorch_models/Model_GIN_classifier_to_explain_17-10-2022_19h_45min_best_ValAcc_0.6688042248208224_epoch_2.pt")
loader_path = "./results/dataloaders/MASKED_loader_Concen_plus_Fluxes.pt"
loader = torch.load(loader_path).get_validation_loader()
explainer  = GNNExplainer(model, epochs=999, num_hops = 5, lr=0.002).to('cuda')
model.to('cuda')

/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device = 'cuda:0'


GIN_classifier_to_explain(
  (GIN_layers): GIN(1, 8, num_layers=4)
  (FC1): Linear(in_features=8, out_features=1, bias=True)
  (FC2): Linear(in_features=12437, out_features=2, bias=True)
  (leakyrelu): LeakyReLU(negative_slope=0.01)
)

In [2]:
a_batch = next(iter(loader)).to('cuda')
model.eval()
gc.collect()
torch.cuda.empty_cache() 
a_batch = next(iter(loader)).to('cuda')
explainer.explain_graph(a_batch.x, a_batch.edge_index)


Explain graph:  13%|█▎        | 132/999 [00:11<01:15, 11.52it/s]

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd

pd.Series(edge_mask.cpu().numpy()).plot.hist(bins=500, logy = True, figsize=(7,2.8), grid=False, color = 'blue', alpha=.8)
plt.xlabel('Edge weight')
plt.tight_layout()
plt.grid(color = 'green', linestyle = '--', linewidth = 0.9, alpha=.5)
#plt.xlim([0, 1])
plt.legend(['Control'])
plt.show()
